## Learning objectives
* Students should be able to assess the structure and cleanliness of their dataset, including size and shape of data, number of variables of each type 
* Students should be able to describe their findings, translate results from code to text using Markdown comments in the Jupyter Notebook, and summarize their thought process in a narrative
* Students should be able to modify the raw data to prepare a clean data set -- including copying data, removing or replacing missing and incoherent data, dropping columns, removing duplicates in Pandas and Jupyter -- and explain and justify their decisions in markdown in their Jupyter notebook
* Students should be able to assess whether their data is “Tidy” and identify appropriate steps and write and  execute code to arrange it into a tidy format - including merging, reshaping, subsetting, grouping, sorting, making appropriate new columns  - and explain and justify their decisions in markdown in their Jupyter notebook
* Students should be able to identify several relevant summary measures, illustrate data using appropriate plots, and explain and justify their decisions in markdown in their Jupyter notebook
* Student should assess the summaries and plots and appraise the need for repeated or further analysis, and justify decisions in markdown


# Describe findings, translate results into Markdown text
 This is more of an overarching goal that should be woven in through the lesson

# Assess the structure and cleanliness

## About Libraries in Python

A library in Python contains a set of tools (called functions) that perform tasks on our data. Importing a library is like getting a piece of lab equipment out of a storage locker and setting it up on the bench for use in a project. Once a library is imported, it can be used or called to perform many tasks.

## The Pandas Library

One of the best options for working with tabular data in Python is to use the Python Data Analysis Library (a.k.a. Pandas). The Pandas library provides data structures, produces high quality plots with matplotlib and integrates nicely with other libraries that use NumPy (which is another Python library) arrays.

Python doesn’t load all of the libraries available to it by default. We have to add an import statement to our code in order to use library functions. To import a library, we use the syntax `import libraryName`. If we want to give the library a nickname to shorten the command, we can add `as nickNameHere`. An example of importing the pandas library using the common nickname pd is below.


In [1]:
import pandas as pd

Each time we call a function that’s in a library, we use the syntax `LibraryName.FunctionName`. Adding the library name with a `.` before the function name tells Python where to find the function. In the example above, we have imported Pandas as `pd`. This means we don’t have to type out pandas each time we call a Pandas function.

We will begin by locating and reading our data which are in a table format. We can use Pandas’ `read_table` function to pull the file directly into a DataFrame.

## What’s a DataFrame?
A DataFrame is a 2-dimensional data structure that can store data of different types (including characters, integers, floating point values, factors and more) in columns. It is similar to a spreadsheet or an SQL table or the data.frame in R. A DataFrame always has an index (0-based). An index refers to the position of an element in the data structure.

Note that we use `pd.read_table`, not just `read_table` or `pandas.read_table`, because we imported pandas as pd.

The columns in the data set are separated by a TAB. We need to tell the `read_table` function in Pandas that that is the case with `sep = ‘\t’`.


In [50]:
url = "https://raw.githubusercontent.com/STAT545-UBC/STAT545-UBC.github.io/master/gapminderDataFiveYear_dirty.txt"
gapminder = pd.read_table(url, sep = "\t")
gapminder.head()

,year,pop,lifeExp,gdpPercap,region
0,1952,8425333.0,28.801,779.445314,Asia_Afghanistan
1,1957,9240934.0,30.332,820.853030,Asia_Afghanistan
2,1962,10267083.0,31.997,853.100710,Asia_Afghanistan
3,1967,11537966.0,34.020,836.197138,Asia_Afghanistan
4,1972,13079460.0,36.088,739.981106,Asia_Afghanistan


### How many rows and columns are in the data?
We often want to know how many rows and columns are in the data  --  we want to know what is called the "shape" attribute of the data frame. Pandas has a convenient way for getting that information by using the `DataFrame.shape` (using DataFrame as a generic name for a, well, data frame). This returns a tuple (values separated by commas) representing the dimensions of the DataFrame (rows, columns).<p>
To get the shape of the gapminder data frame:

In [4]:
gapminder.shape

(1704, 5)

The `info()` method gives a few useful pieces of information, including the shape of the dataframe, the variable type of each column, and the amount of memory stored.

In [22]:
gapminder.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1704 entries, 0 to 1703
Data columns (total 5 columns):
year         1704 non-null int64
pop          1704 non-null float64
lifeExp      1704 non-null float64
gdpPercap    1704 non-null float64
region       1704 non-null object
dtypes: float64(3), int64(1), object(1)
memory usage: 66.6+ KB


The `describe()` method will take the numeric columns and give a summary of their values. This is useful for getting a sense of the ranges of values and seeing if there are any unusual or suspicious numbers.


In [5]:
gapminder.describe()

,year,pop,lifeExp,gdpPercap
count,1704.00000,1.704000e+03,1704.000000,1704.000000
mean,1979.50000,2.960121e+07,59.474439,7215.327081
std,17.26533,1.061579e+08,12.917107,9857.454543
min,1952.00000,6.001100e+04,23.599000,241.165877
25%,1965.75000,2.793664e+06,48.198000,1202.060309
50%,1979.50000,7.023596e+06,60.712500,3531.846989
75%,1993.25000,1.958522e+07,70.845500,9325.462346
max,2007.00000,1.318683e+09,82.603000,113523.132900



describe() just blindly looks at all numeric variables. But we wouldn't actually want to take the mean year. Let's pull out only the correct columns.

In [17]:
gapminder[['pop', 'lifeExp', 'gdpPercap']].describe()

,pop,lifeExp,gdpPercap
count,1.704000e+03,1704.000000,1704.000000
mean,2.960121e+07,59.474439,7215.327081
std,1.061579e+08,12.917107,9857.454543
min,6.001100e+04,23.599000,241.165877
25%,2.793664e+06,48.198000,1202.060309
50%,7.023596e+06,60.712500,3531.846989
75%,1.958522e+07,70.845500,9325.462346
max,1.318683e+09,82.603000,113523.132900


The command value_counts() gives you a first global idea of your categorical data such as strings. In this case that is the column ‘Region’.
The table reveals some problems. The data set covers 12 years, so each ‘region’ should appear 12 times. We also see inconsistencies (string variables are very susceptible to those), for instance:

Asia_china	vs. Asia_China

Another type of problem is the various names of Congo. In order to analyze this dataset appropriately we need to take care of these issues.

In [16]:
print(len(gapminder['region'].unique())) # How many unique regions are in the data?
gapminder['region'].value_counts() # How many times does each unique region occur?

151


Asia_Oman                                  12
Asia_Cambodia                              12
Africa_Namibia                             12
Oceania_New Zealand                        12
Africa_Gabon                               12
Europe_Czech Republic                      12
Africa_Nigeria                             12
Africa_Sao Tome and Principe               12
Africa_Mali                                12
Africa_Guinea                              12
Asia_Bangladesh                            12
Africa_Chad                                12
Africa_Kenya                               12
Asia_Nepal                                 12
Africa_Zambia                              12
Europe_Sweden                              12
Asia_Iraq                                  12
Americas_Bolivia                           12
Africa_Mauritania                          12
Europe_Norway                              12
Asia_Syria                                 12
Asia_Yemen, Rep.                  

This table reveals some problems: we should have 12 counts for every country/region but some have fewer than 12. E.g. _Canada, Asia_China vs Asia_china, etc. It will require some string processing to clean up.

<font color='red'>**TO DO:** create an exercise</font>

# Modify the dataset -- cleaning

Skills to illustrate
* Make a copy of the data
in place vs. not
pandas.DataFrame.copy() - make a copy of data frame
* Assigning to new variable/df names 
* Dealing with missing data
Pros and cons of dropping NAs and inconsistent data
* Dealing with incoherent data (NA, na, N/A, n/a, ND, not done, XXXX) misspellings, etc
Regex, data transformation to address inconsistency (fill_na)
* dropping columns `df.drop()`:
_In many instances, data sets may include variables that might not be relevant 
or interesting for the intended data exploration or downstream analysis. After
an initial examination of the data, it's generally useful to remove from the 
data set those columns that correspond to irrelevant variables. Whenever 
removing variables, it is good practice to note exactly why such columns are
being dropped._
* Removing duplicates
df.drop_duplicates()

## New variables: country and continent

The `region` column is a bit too messy for what we'd like to do.
**TO DO:** narrative about tidy data

### String manipulations

Very common problems with string variables are  lingering white space and upper case vs. lower case.
The following three commands remove all such trailing spaces (left and right) and put everything in lowercase. If you prefer, the three commands can be written in one single line (concept: chaining). 

In [52]:
gapminder['region'] = gapminder['region'].str.lstrip() # Strip white space on left
gapminder['region'] = gapminder['region'].str.rstrip() # Strip white space on right
gapminder['region'] = gapminder['region'].str.lower() # Convert to lowercase
gapminder['region'].value_counts() # How many times does each unique region occur?

# We could have done this in one line!
# gapminder['region'] = gapminder['region'].str.lstrip().str.rstrip().lower()

asia_iraq                                  12
americas_peru                              12
europe_montenegro                          12
africa_morocco                             12
africa_south africa                        12
europe_bosnia and herzegovina              12
asia_japan                                 12
africa_ghana                               12
africa_kenya                               12
africa_sao tome and principe               12
africa_equatorial guinea                   12
europe_france                              12
africa_mauritania                          12
asia_india                                 12
europe_italy                               12
asia_malaysia                              12
americas_united states                     12
asia_thailand                              12
europe_poland                              12
africa_nigeria                             12
europe_slovenia                            12
americas_trinidad and tobago      

### regex + replace()
**Note: In code we need to uncomment correct regex of congo**
A regular expression, aka regex, is a sequence of characters that define a search pattern. In a regular expression, the symbol “*” matches the preceding character 0 or more times, whereas “+” matches the preceding character 1 or more times. “.” matches any single character. Writing “x|y” means to match either ‘x’ or ‘y’.

For more regex shortcuts (cheatsheet): https://www.shortcutfoo.com/app/dojos/regex/cheatsheet

Pandas allows you to use regex in its replace() function -- when a regex term is found in an element, the element is then replaced with the specified replacement term. In order for it to appropriately correct elements, both regex and inplace variables need to be set to True (as their defaults are false). This ensures that the initial input string is read as a regular expression and that the elements will be modified in place.

For more documentation on the replace method: http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.replace.html

Here's an incorrect regex example: we create a temporary DataFrame in which a regex pulls all values that contain the term “congo”. Unfortunately, this creates 24 instances of the Democratic Republic of the Congo -- this is an error in our cleaning! We can revert back to the non-temporary DataFrame and correctly modify our regex to isolate only the Democratic Republic instances (as opposed to including the Republic as well).

In [56]:
# This gives a problem -- 24 values of the congo!
gapminder[gapminder["region"].str.contains('congo')]
temp = gapminder['region'].replace(".*congo.*", "africa_dem rep congo", regex=True)
temp.value_counts()

africa_dem rep congo             24
asia_iraq                        12
africa_zimbabwe                  12
africa_morocco                   12
africa_south africa              12
europe_bosnia and herzegovina    12
asia_japan                       12
africa_ghana                     12
africa_kenya                     12
africa_sao tome and principe     12
africa_equatorial guinea         12
europe_france                    12
africa_mauritania                12
asia_india                       12
europe_italy                     12
asia_malaysia                    12
americas_united states           12
europe_montenegro                12
asia_thailand                    12
europe_poland                    12
asia_pakistan                    12
americas_guatemala               12
africa_gabon                     12
americas_puerto rico             12
asia_lebanon                     12
africa_egypt                     12
europe_slovenia                  12
europe_portugal             

In [58]:
gapminder[gapminder["region"].str.contains('congo')]
gapminder['region'].replace(".*congo, dem.*", "africa_dem rep congo", regex=True, inplace=True)
gapminder['region'].replace(".*_democratic republic of the congo", "africa_dem rep congo", regex=True, inplace=True)
gapminder['region'].value_counts()

asia_iraq                        12
africa_sao tome and principe     12
africa_morocco                   12
africa_south africa              12
europe_bosnia and herzegovina    12
asia_japan                       12
africa_ghana                     12
africa_kenya                     12
africa_equatorial guinea         12
asia_thailand                    12
europe_france                    12
africa_mauritania                12
asia_india                       12
europe_italy                     12
asia_malaysia                    12
americas_united states           12
europe_montenegro                12
europe_poland                    12
africa_zimbabwe                  12
asia_pakistan                    12
americas_guatemala               12
africa_gabon                     12
americas_puerto rico             12
asia_lebanon                     12
africa_egypt                     12
europe_slovenia                  12
europe_portugal                  12
africa_nigeria              

In [59]:
gapminder['region'].replace(".*ivore.*", "africa_cote d'ivoire", regex=True, inplace=True)
gapminder['region'].replace("^_canada", "americas_canada", regex=True, inplace=True)
gapminder['region'].value_counts()

asia_iraq                        12
africa_kenya                     12
europe_montenegro                12
africa_morocco                   12
africa_south africa              12
europe_bosnia and herzegovina    12
asia_japan                       12
africa_ghana                     12
africa_sao tome and principe     12
europe_poland                    12
africa_equatorial guinea         12
europe_france                    12
africa_mauritania                12
asia_india                       12
europe_italy                     12
asia_malaysia                    12
asia_thailand                    12
africa_nigeria                   12
americas_bolivia                 12
europe_slovenia                  12
americas_guatemala               12
africa_gabon                     12
africa_cote d'ivoire             12
americas_puerto rico             12
asia_lebanon                     12
africa_egypt                     12
asia_pakistan                    12
europe_finland              

## Tidy data

Having what is called a _Tidy_ data set can make cleaning your data much easier. Two of the important aspects of Tidy data are:
* every variable has its own column
* every observation has its own row

(there are other aspects of Tidy data, here is a good blog post about Tidy data in Python: http://www.jeannicholashould.com/tidy-data-in-python.html)

Currently the dataset has a single column for continent and country (the ‘region’ column). We can split that column into two, by using the underscore that separates continent from country.
We can create a new column in the DataFrame by naming it before the = sign:
`gapminder[‘country’] = `

The following commands use the function split() to split the string at the underscore (the first argument), which results in a list of two elements: before and after the \_. The second argument tells split() that the split should take place only at the first occurrence of the underscore.

In [ ]:
gapminder['country']=gapminder['region'].str.split('_', 1).str[1]
gapminder['continent']=gapminder['region'].str.split('_', 1).str[0]
gapminder


## Handling Missing Data
If you find that your data set contains missing data, pandas allows you to either remove all instances will null data or replace them with a particular value.

`df = df.dropna()` drops rows with any column having NA/null data
`df = df.fillna(value)` %replace all NA/null data with value

## Handling (Unwanted) Repetitive Data
You can identify which observations are duplicates.
The call `df.duplicated()` will return boolean values for each row in the DataFrame telling you whether or not a row is repetitive.

In cases where you don’t want repetitive values (we wouldn’t--we only want each country to be represented once for every relevant year), you can easily drop such duplicate rows with the call `df.drop_duplicates()`.

## Referencing objects vs copying objects
Suppose you take a subset of your data and store it in a new variable, like `gapminder_early = gapminder[gapminder['year'] < 1970]`.  Doing this does not actually create a new object. Instead, you have just given a name to that subset of the original data: gapminder_early points to those rows of gapminder.  Any changes you make to the new dataframe gapminder_early will appear in the corresponding rows of gapminder too.  

Often, you want to leave the original data untouched.  To avoid this problem, you can make a copy of your data before operating on a subset of the data.  This will ensure that one version of the original data is preserved.  For instance:

<font color='red'>**TO DO:** this example doesn't actually illustrate the problem :(</font>

In [75]:
gapminder_copy = gapminder.copy()
gapminder_early = gapminder_copy[gapminder_copy['year'] < 1970]
gapminder_early['pop'] = 0
#print(gapminder_copy[gapminder_copy['year'] < 1970].head())
#print(gapminder[gapminder['year'] < 1970].head())

/Library/Frameworks/Python.framework/Versions/3.4/lib/python3.4/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


## Prepare the data structure -- tidy data

* Describe Tidy Data - each variable has it’s own column, each observation has its own row. 
* Reshaping (if necessary) 
renaming columns (from names that don’t make sense, names with strange characters, etc to names that make sense) 
* Merging datasets,
Pandas - merge()[left, right, inner, outer], concat()
* Subsetting data
0) refer back to the way we selected columns in .describe() above
1) Indexing - numerically(zero indexing) vs by names, boolean indices, 
2) Slicing - base python and/or pandas
3) List comprehensions
* Regular expressions (complicated! But useful and important)
* Grouping data, Indexing DataFrames
df.groupby(by="col")
* Sorting data
df.sort_values()
df.sort_index()
* Creating new variables/columns (for transformation (log, sqrt, etc)

Ideas: subset by continent

## Summarize and plot

Summaries (but can’t *say* statistics…)
* Sort data
* Can make note about using numpy functions, dif between dataframe and array
Good Plots for the data/variable type



Plots 
* of subsets, 
* single variables
* pairs of variables
* Matplotlib syntax (w/ seaborn for defaults (prettier, package also good for more analysis later...))

Exploring is often iterative - summarize, plot, summarize, plot, etc. - sometimes it branches…


## Interpret plots